In [1]:
import os
import hashlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from galaxy_datasets.shared import label_metadata


In [2]:
dr12 = pd.read_parquet('/media/walml/mirror/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr12_ortho_v5_labelled_catalog.parquet')
dr5 = pd.read_parquet('/media/walml/mirror/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr5_ortho_v5_labelled_catalog.parquet')
dr8 = pd.read_parquet('/media/walml/mirror/galaxy_zoo/decals/dr8/catalogs/training_catalogs/dr8_ortho_v5_labelled_catalog.parquet')

In [3]:
dr12

,iauname,in_gzd_c,data_release,smooth-or-featured-dr12_total-votes,smooth-or-featured-dr12_smooth,smooth-or-featured-dr12_smooth_fraction,smooth-or-featured-dr12_featured-or-disk,smooth-or-featured-dr12_featured-or-disk_fraction,smooth-or-featured-dr12_artifact,smooth-or-featured-dr12_artifact_fraction,...,spiral-arm-count-dr5_3_fraction,spiral-arm-count-dr5_4_fraction,spiral-arm-count-dr5_more-than-4_fraction,spiral-arm-count-dr5_cant-tell_fraction,merging-dr5_none_fraction,merging-dr5_minor-disturbance_fraction,merging-dr5_major-disturbance_fraction,merging-dr5_merger_fraction,dr_with_most_votes,id_str
0,J161835.06+094825.4,False,DR1,47,29.0,0.617021,8.0,0.170213,10.0,0.212766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,387274_3520
1,J171558.33+294918.8,False,DR2,36,22.0,0.611111,4.0,0.111111,10.0,0.277778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,495330_3526
2,J152604.39+140215.0,True,DR2,38,28.0,0.736842,8.0,0.210526,2.0,0.052632,...,NaN,NaN,NaN,NaN,0.800000,0.0,0.0,0.200000,dr12,411197_3292
3,J074034.26+240659.4,False,DR2,38,17.0,0.447368,14.0,0.368421,7.0,0.184211,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,465235_5856
4,J161921.36+131433.9,False,DR2,35,29.0,0.828571,1.0,0.028571,5.0,0.142857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,407046_2961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82355,J082618.08+111227.6,False,DR2,37,28.0,0.756757,2.0,0.054054,7.0,0.189189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,395315_1473
82356,J225716.71+000240.9,False,DR1,44,35.0,0.795455,5.0,0.113636,4.0,0.090909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,331745_4989
82357,J154806.17+142227.1,True,DR2,29,10.0,0.344828,17.0,0.586207,2.0,0.068966,...,NaN,NaN,NaN,NaN,0.833333,0.0,0.0,0.166667,dr12,412616_3932
82358,J001510.54-010434.1,False,DR1,49,39.0,0.795918,4.0,0.081633,6.0,0.122449,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dr12,324623_1219


In [4]:
# dr12, dr5 and dr8 all now have id_str/dr8_id, so I don't need this any more?


# megacatalog = pd.read_parquet('/home/walml/repos/decals-rings/results/rings_megacatalog.parquet', columns=['dr8_id', 'iauname', 'redshift'])
# megacatalog.sample(5)

In [5]:
# print(len(dr12))
# dr12 = pd.merge(dr12, megacatalog, on='iauname', how='inner')
# print(len(dr12))
# # 85730
# # 82387

In [6]:
# print(len(dr5))
# dr5 = pd.merge(dr5, megacatalog, on='iauname', how='inner')
# print(len(dr5))
# # 228059
# # 222754

In [7]:
labelled = pd.concat([dr12, dr5, dr8], axis=0)
len(labelled)
# 401250

401250

In [8]:
unlabelled = pd.read_parquet('/media/walml/mirror/galaxy_zoo/decals/dr8/catalogs/data_release/master_all_file_index_passes_file_checks.parquet', columns=['ra', 'dec', 'dr8_id', 'brickid', 'objid', 'jpg_loc'])
unlabelled['id_str'] = unlabelled['dr8_id'].astype(str)
unlabelled['brickid'] = unlabelled['brickid'].astype(str)
unlabelled['objid'] = unlabelled['objid'].astype(str)
# for col in decals_all_campaigns_ortho_label_cols:
#     unlabelled[col] = 0
# unlabelled = unlabelled.copy().sample(len(unlabelled), random_state=42)

In [9]:
unlabelled.head()

,ra,dec,dr8_id,brickid,objid,jpg_loc,id_str
0,32.084931,-44.311422,100000_1081,100000,1081,/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...,100000_1081
1,32.140085,-44.293668,100000_1401,100000,1401,/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...,100000_1401
2,32.275015,-44.288957,100000_1483,100000,1483,/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...,100000_1483
3,32.045648,-44.287172,100000_1509,100000,1509,/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...,100000_1509
4,32.170627,-44.267273,100000_1869,100000,1869,/share/nas2/walml/galaxy_zoo/decals/dr8/jpg/10...,100000_1869


In [10]:
for df in [labelled]:  # , unlabelled dr8 already has but it points to the png
    df['dr8_id'] = df['id_str'] # ensure consistency
    df['brickid'] = df['id_str'].apply(lambda x: x.split('_')[0])
    df['objid'] = df['id_str'].apply(lambda x: x.split('_')[1])
for df in [labelled, unlabelled]:
    df['subfolder'] = df['brickid']
    df['filename'] = df['id_str'] + '.jpg'
    df['file_loc'] = '/home/walml/data/desi/jpg/' + df['brickid'] + '/' + df['filename']
    df = df.copy()  # defragment
 
    print(df['file_loc'][0], df['filename'][0])

/tmp/ipykernel_666493/3409716909.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['brickid'] = df['id_str'].apply(lambda x: x.split('_')[0])
/tmp/ipykernel_666493/3409716909.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['objid'] = df['id_str'].apply(lambda x: x.split('_')[1])
/tmp/ipykernel_666493/3409716909.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axi

0    /home/walml/data/desi/jpg/387274/387274_3520.jpg
0    /home/walml/data/desi/jpg/302281/302281_3952.jpg
0    /home/walml/data/desi/jpg/107510/107510_4476.jpg
Name: file_loc, dtype: object 0    387274_3520.jpg
0    302281_3952.jpg
0    107510_4476.jpg
Name: filename, dtype: object
/home/walml/data/desi/jpg/100000/100000_1081.jpg 100000_1081.jpg


In [11]:
# labelled is missing ra/dec for many galaxies - merge and overwrite
del labelled['ra']
del labelled['dec']
labelled = pd.merge(labelled, unlabelled[['id_str', 'ra', 'dec']], on='id_str', how='inner', validate='one_to_one')

In [12]:
label_cols = label_metadata.decals_all_campaigns_ortho_label_cols

np.any(labelled[label_cols].isna())
# any(labelled[label_cols].isna()) doesn't work right

False

In [13]:
np.any(labelled[label_cols].isna(), axis=1).mean()
# any(labelled[label_cols].isna().values.max(axis=1)) doesn't work right

0.0

In [14]:
train_catalog, test_catalog = train_test_split(labelled, test_size=0.2, random_state=42)  # note that because we dropped a few rows which couldn't be cross-matched, this is not the same split as the original_images version

In [15]:
train_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/gz_desi/gz_desi_train_catalog.parquet'
test_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/gz_desi/gz_desi_test_catalog.parquet'
unlabelled_catalog_loc = '/home/walml/repos/galaxy-datasets/roots/desi_unlabelled/desi_unlabelled_catalog.parquet'

train_catalog = train_catalog.reset_index(drop=True)
test_catalog = test_catalog.reset_index(drop=True)
unlabelled = unlabelled.reset_index(drop=True)

train_catalog.to_parquet(train_catalog_loc, index=False)
test_catalog.to_parquet(test_catalog_loc, index=False)

# this will not be uploaded as images, wayyyy to big
unlabelled.to_parquet(unlabelled_catalog_loc, index=False)

/home/walml/miniforge3/envs/rings/lib/python3.9/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [16]:

for loc in [
    train_catalog_loc,
    test_catalog_loc,
    unlabelled_catalog_loc
]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

edcbbe4f0adb767d26bdc5de26aa854d
95bfed06c5a00742f58171f46cb1d8c1
5e270f18516e2e4be9786c11cfcc6a9c


In [17]:

# set file_locs, relative to rsync root
all_files = list('jpg/' + labelled['brickid'] + '/' + labelled['id_str'] + '.jpg')

with open('/home/walml/repos/galaxy-datasets/derived_data/gz_desi_all_files.txt', 'w') as f:
    f.writelines([x + '\n' for x in all_files])


In [18]:
# and now run the code in list_of_images_to_chunks

In [19]:
import glob

locs_to_checksum = glob.glob(f'/home/walml/data/desi/gz_desi_chunks/*_chunk_0*.tar.gz')
assert locs_to_checksum
locs_to_checksum.sort()

for loc in locs_to_checksum:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(os.path.basename(loc), md5_checksum)

gz_desi_chunk_00_archive.tar.gz 76c861762633d022603686e0794ce99c
gz_desi_chunk_01_archive.tar.gz 844502e49d6e087b6c1f4fcfef0772c0
gz_desi_chunk_02_archive.tar.gz b793771b8fc9383b616d2dc67bdfbf7e
gz_desi_chunk_03_archive.tar.gz 003821161b4990a49d1dcb5d72d49b12
gz_desi_chunk_04_archive.tar.gz aa85dec158d95edea98b31efe30231d5
gz_desi_chunk_05_archive.tar.gz 36ed5893441a533d1d74fe480cb42cb2
gz_desi_chunk_06_archive.tar.gz 6b84e92c7b4f1edbb372d2247ff1740a
gz_desi_chunk_07_archive.tar.gz feef448b45eeb3d5edc3f44d936f386a
gz_desi_chunk_08_archive.tar.gz c8be005044f94046a7080617815a3d00
